<a href="https://colab.research.google.com/github/davidharrisnet/marvel_universe/blob/main/HelloLlamaLocal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#https://github.com/facebookresearch/llama-recipes/blob/main/demo_apps/HelloLlamaLocal.ipynb

## This demo app shows:
* How to run Llama2 locally on a Mac using llama-cpp-python and the llama-cpp's quantized Llama2 model
* How to use LangChain to ask Llama general questions
* How to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and ask questions about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination

We start by installing necessary requirements and import packages we will be using in this example.
- [llama-cpp-python](https://github.com/abetlen/llama-cpp-python) a simple Python bindings for [llama.cpp](https://github.com/ggerganov/llama.cpp) library
- pypdf gives us the ability to work with pdfs
- sentence-transformers for text embeddings
- chromadb gives us database capabilities
- langchain provides necessary RAG tools for this demo

In [6]:
# install all the required packages for the demo
!pip install llama-cpp-python


In [7]:
!pip install pypdf sentence-transformers chromadb langchain --quiet

In [8]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [9]:
import os

Next, initialize the langchain `CallBackManager`. This handles callbacks from Langchain and for this example we will use token-wise streaming so the answer gets generated token by token when Llama is answering your question.

In [10]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


Set up the Llama 2 model.

Replace `<path-to-llama-gguf-file>` with the path either to your downloaded quantized model file [here](https://drive.google.com/file/d/1afPv3HOy73BE2MoYCgYJvBDeQNa9rZbj/view?usp=sharing),

or to the `ggml-model-q4_0.gguf` file built with the following commands:

```bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
python3 -m pip install -r requirements.txt
python convert.py <path_to_your_downloaded_llama-2-13b_model>
./quantize <path_to_your_downloaded_llama-2-13b_model>/ggml-model-f16.gguf <path_to_your_downloaded_llama-2-13b_model>/ggml-model-q4_0.gguf q4_0

```
For more info see https://python.langchain.com/docs/integrations/llms/llamacpp

In [11]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [12]:
model_path = os.path.join("/content","drive","My Drive", "models", "ggml-model-q4_0.gguf")


In [13]:

llm = LlamaCpp(
    model_path=model_path,
    temperature=0.0,
    top_p=1,
    n_ctx=6000,
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


With the model set up, you are now ready to ask some questions.

Here is an example of the simplest way to ask the model some general questions.

In [14]:

question = "who wrote the book Innovator's dilemma?"
answer = llm(question)



The book "The Innovator's Dilemma" was written by Clayton Christensen, a professor at Harvard Business School. It was first published in 1997 and has since become a widely influential book on business strategy and innovation.

Alternatively, you can sue LangChain's `PromptTemplate` for some flexibility in your prompts and questions.

For more information on LangChain's prompt template visit this [link](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)

In [15]:
# a more flexible way to ask Llama general questions using LangChain's PromptTemplate and LLMChain
prompt = PromptTemplate.from_template(
    "who wrote {book}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.run("innovator's dilemma")

Llama.generate: prefix-match hit




Clayton Christensen is the author of "The Innovator's Dilemma," which was first published in 1997. The book explores why successful companies often struggle to adapt to disruptive technologies and business models that ultimately lead to their downfall. Christensen argues that these companies are faced with a dilemma because they have invested so heavily in their existing products and processes that it is difficult for them to pivot and embrace new, disruptive technologies. He identifies three types of innovation - sustaining, disruptive, and transformative - and explains how companies can use these different types of innovation to stay ahead of the competition. The book has had a significant impact on business strategy and has been widely cited as a key influence on the way companies approach innovation.

Now, let's see how Llama2 hallucinates, because it did not have knowledge about Llama2 at the time it was trained.
By default it behaves like a know-it-all expert who will not say "I don't know".

In [16]:

prompt = PromptTemplate.from_template(
    "What is {what}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.run("llama2")

Llama.generate: prefix-match hit




Llama2 is a free, open-source tool for generating high-quality, randomized test data for software applications. It is designed to be easy to use and to produce realistic, diverse test data that can help you identify and fix bugs in your application before it is released.

Llama2 is the successor to the popular Llama tool, which was developed by the University of California, Los Angeles (UCLA) and was widely used in the software testing industry. Llama2 builds on the success of Llama and adds several new features and improvements, including support for multiple data types, a more powerful and flexible API, and improved performance.

Llama2 is written in Java and is available under the Apache License, which means that it is free to use, modify, and distribute. It can be run on any platform that has a Java Virtual Machine (JVM) installed, including Windows, macOS, and Linux.

What are some of the key features of llama2?

Some of the key features of llama2 include:

1. Support for multip

In [17]:
pdf_path = os.path.join("/content","drive","My Drive","Colab Notebooks", "docs", "llama2.pdf")

In [18]:
os.path.exists(pdf_path)

True

One way we can fix the hallucinations is to use RAG, to augment it with more recent or custom data that holds the information for it to answer correctly.

First we load the Llama2 paper using LangChain's [PDF loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

In [19]:

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [20]:
# quick check on the loaded document for the correct pages etc
print(len(documents), documents[0].page_content[0:300])

77 Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernande


Next we will store our documents.
There are more than 30 vector stores (DBs) supported by LangChain.
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

We will also import the `HuggingFaceEmbeddings` and `RecursiveCharacterTextSplitter` to assist in storing the documents.

In [21]:

from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database.


In [22]:
# split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)


We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.

In [23]:
# use another LangChain's chain, RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)



For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to the model to answer the question.

It takes close to 2 minutes to return the result (but using other vector stores other than Chroma such as FAISS can take longer) because Llama2 is running on a local Mac.
To get much faster results, you can use a cloud service with GPU used for inference - see HelloLlamaCloud for a demo.

In [24]:

question = "What is llama2?"
result = qa_chain({"query": question})


Llama.generate: prefix-match hit


 Llama 2 is a new language model developed by Meta AI that has the potential to generate inaccurate or objectionable responses to user prompts. It is not recommended to use this model without proper safety testing and tuning, as it may produce harmful or inappropriate outputs. The responsible use guide and code examples are available on the Meta AI website for developers who wish to use this model safely and responsibly.